# Linear Regression Multi Dimension + SGD Study

8-10-2018
Setup

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def loss(yt, y):
    return np.mean((yt - y)**2)/2

generate a bunch of random points

In [11]:
m = np.random.randn(2)
x = np.random.randn(2,1000) * 10
b = np.random.randn()
mtrue = np.random.randn(1,2)
y = mtrue @ x + b + np.random.randn(2,1000)